# Student Details

***

**Name:** Hoai Nhan Nguyen <br>
**Student Number:** sba24098 <br>
**Course:** Higher Diploma in Science in Artificial Intelligence Applications 

***

# Assessment Task

Students are advised to review and adhere to the submission requirements documented after the assessment task.

# Assessment details

You are required to use the dataset attached “mye22final.xlsx”, that contains data on Estimates of the population for the UK, England, Wales, Scotland, and Northern Ireland for the years 2011 and 2022.

This Data is Licenced under open-government-licence v3.

https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/

Create an interactive Dashboard aimed at younger adults (18 - 35 years).

The dashboard may be created using ANY pythonic method within a Jupyter Notebook. PYTHON FILES May be used after they have been generated from the Jupyter Notebook or as helper files, but the main work MUST be completed in Jupyter Notebook. (You may (if you wish) Host the Dashboard using STREAMLIT or Similar). The Dashboard may be viewed in HTML format, within the notebook or as a standalone application based on the techniques covered in the class, (Tkinter, panel, dash, Jupyter-dash etc…). All final code and files MUST be uploaded to MOODLE and progress shown in your GitHub Classroom commits.

<b>YOU MAY NOT USE POWERBI, TABLEAU, LOOKER, or any other generalized tool for this assignment.</b>

The Dashboard must detail the following requirements:

* Be Interactive and include Hover functionality.
* Contain at least 2 Visualizations.
* Display population densities by Geographic location.
* Allow the user to select and display population densities by age AND by Gender.
* Display a comparison between the 2011 figures and the 2022 figures.

(70 Marks)

Discuss in detail your rationale and justification for all stages of data preparation for your visualizations and how your dashboard is designed with this demographic (younger adults (18 - 35 years)) in mind.  

(30 marks)
***

## Data Analysis and Preprocessing

**Importing libraries**

In [ ]:
import pandas as pd

**Loading Excel file and printing sheet names**

In [ ]:
# Load the Excel file
xls = pd.ExcelFile("data/mye22final.xlsx")

# Print the names of all sheets
print(xls.sheet_names)

**Reviewing information on MYEB1 and MYE5 sheet in the Excel File**

In [ ]:
df_uk_population = pd.read_excel('data/mye22final.xlsx', sheet_name='MYEB1')
df_uk_density = pd.read_excel('data/mye22final.xlsx', sheet_name='MYE5', skiprows=7)

In [ ]:
# Check the head of the dataset
df_uk_population.head()

In [ ]:
# Check the head of the dataset
df_uk_density.head()

**Checking the data type of dataframe**

In [ ]:
df_uk_population.info()

In [ ]:
df_uk_density.info()

**Adjusting the column names for df_uk_density dataframe**

In [ ]:
df_uk_density.columns = [
    'code',
    'name',
    'geography',
    'area_sq_km',
    'population_2022',
    'person_per_sq_km_2022',
    'population_2011',
    'person_per_sq_km_2011'
]


**Checking any null or duplicate rows in dataframe.**

In [ ]:
def checking_null_and_duplicate(df):
    if df.isnull().values.any():
        print("There is a null value in the DataFrame.")
    else:
        print("There is no null value in the DataFrame.")    
    
    # Check for duplicates
    if df.duplicated().sum() > 0:
        print("There is a duplicate value in the DataFrame.\n")
    else:
        print("There is no duplicate value in the DataFrame.\n")    

# Checking MYEB1 Sheet
print("== MYEB1 Sheet ==")
checking_null_and_duplicate(df_uk_population)

# Checking MYE5 Sheet
print("== MYE5 Sheet ==")
checking_null_and_duplicate(df_uk_density)



**Checking the uniqueness of the dataframe**

In [ ]:
df_uk_population.nunique()


In [ ]:
df_uk_density.nunique()

**Checking Unique values for the column "name".**

In [ ]:
df_uk_population['name'].unique()

In [ ]:
df_uk_density['name'].unique()

**Checking Unique values for the column "geography".**

In [ ]:
df_uk_population['geography'].unique()

In [ ]:
df_uk_density['geography'].unique()

# Feature Engineering Dataframes

### df_uk_population dataframe

**Filtering the dataframe based on the geography types for the visualisation**

In [ ]:
# List of geography types to keep for the visualisation
geographies = [
    'Unitary Authority',
    'London Borough',
    'Metropolitan District',
    'Non-metropolitan District',
    'Council Area',
    'Local Government District'
]

# Filtering the dataframes to only use the above geography types
df_uk_population = df_uk_population[df_uk_population['geography'].isin(geographies)]
# Saving the dataframe as a CSV file
df_uk_population.to_csv("data/df_uk_population.csv", index=False)

### df_uk_density dataframe 

**Filtering the dataframe based on the geography types for the visualisation**

In [ ]:
# Filtering the dataframes to only use the above geography types
df_uk_density = df_uk_density[df_uk_density['geography'].isin(geographies)]

**Aggregating and reshaping population data by gender and year for each location code**

In [ ]:
# Summing the populations for all ages for each code and sex
gender_populations = df_uk_population.groupby(['code', 'sex'])[['population_2011', 'population_2022']].sum().reset_index()

# Pivot the data to have 'code' as rows and 'sex' as columns for 2011 and 2022 population values
gender_populations = gender_populations.pivot(index='code', columns='sex', values=['population_2011', 'population_2022'])
# Rename the columns to make them more intuitive (2011/2022 male/female population)
gender_populations.columns = ['male_population_2011', 'female_population_2011', 'male_population_2022', 'female_population_2022']
# Reset the index so that 'code' becomes a regular column, not an index
gender_populations = gender_populations.reset_index()

# Reviewing the head of the dataframe.
gender_populations.head()

**Merging gender-specific population data with UK population density data by location code**

In [ ]:
# Merging the dataframes df_uk_density and gender_populations based on location code
df_uk_density_with_gender_population = df_uk_density.merge(gender_populations, how='left', left_on='code', right_on='code')

**Calculating gender-specific population densities per square kilometer for 2011 and 2022**

In [ ]:
# Creating new columns based on the gender_populations gender-specific population densities per square kilometer
df_uk_density_with_gender_population['male_per_sq_km_2022'] = df_uk_density_with_gender_population['male_population_2022'] / df_uk_density_with_gender_population['area_sq_km']
df_uk_density_with_gender_population['female_per_sq_km_2022'] = df_uk_density_with_gender_population['female_population_2022'] / df_uk_density_with_gender_population['area_sq_km']
df_uk_density_with_gender_population['male_per_sq_km_2011'] = df_uk_density_with_gender_population['male_population_2011'] / df_uk_density_with_gender_population['area_sq_km']
df_uk_density_with_gender_population['female_per_sq_km_2011'] = df_uk_density_with_gender_population['female_population_2011'] / df_uk_density_with_gender_population['area_sq_km']

In [ ]:
# Reviewing the head of the dataframe.
df_uk_density_with_gender_population.head()

**Assigning age groups to population data using defined bins and labels**

In [ ]:
# Define age group bins and labels
bins = [0, 5, 12, 18, 39, 59, float('inf')]  # Age group bins
labels = ['early_childhood', 'middle_childhood', 'teens', 'young_adults', 'middle_aged_adults', 'seniors_elderly'] 

# Create a new column 'age_group' based on the 'age' column
df_uk_population['age_group'] = pd.cut(df_uk_population['age'], bins=bins, labels=labels, right=True)

# Fill the NaN values with 'Early Childhood' related to age zero
df_uk_population['age_group'] = df_uk_population['age_group'].fillna('early_childhood')

**Grouping and summing population data by code, sex, and age group**

In [ ]:
# Grouping and summing population data by code, sex, and age group
age_group_summary = df_uk_population.groupby(['code', 'sex', 'age_group'])[['population_2011', 'population_2022']].sum().reset_index()


**Pivoting and formatting age group population data by gender and year for each region code**

In [ ]:
# Replace gender abbreviations with full names
age_group_summary['sex'] = age_group_summary['sex'].map({'F': 'female', 'M': 'male'})

# Pivot the data to have separate columns for each gender and age group
age_group_population = age_group_summary.pivot_table(
    index='code',
    columns=['sex', 'age_group'],  
    values=['population_2011', 'population_2022'],  
    aggfunc='sum'
)

# Flatten the multi-level column names to make them more readable and convert to lowercase
age_group_population.columns = [
    # <gender>_<age_group>_population_<year>
    f'{col[1]}_{col[2]}_{col[0]}' 
    for col in age_group_population.columns
]

# Resetting the index of the dataframe
age_group_population = age_group_population.reset_index()

# Reviewing the head of the dataframe.
age_group_population.head()

**merging age group population data with existing gender and density dataset by location code**

In [ ]:
# Merging the dataframes df_uk_density_with_gender_population and age_group_population based on location code
df_uk_density_with_gender_and_age_population = df_uk_density_with_gender_population.merge(age_group_population, how='left', left_on='code', right_on='code')

In [ ]:
# Reviewing the head of the dataframe.
df_uk_density_with_gender_and_age_population.head()

**Calculating gender-specific age group population densities per square kilometer for 2011 and 2022**

In [ ]:
# Defining age group in the dataframe
age_groups = [
    'early_childhood',
    'middle_childhood',
    'teens',
    'young_adults',
    'middle_aged_adults',
    'seniors_elderly'
]

# For loop to create a column based on the age group and area_sq_km for males in 2022
for group in age_groups:
    pop_col = f'male_{group}_population_2022'
    density_col = f'male_{group}_per_sq_km_2022'
    df_uk_density_with_gender_and_age_population[density_col] = (
        df_uk_density_with_gender_and_age_population[pop_col] /
        df_uk_density_with_gender_and_age_population['area_sq_km']
    )
    
# For loop to create a column based on the age group and area_sq_km for males in 2011
for group in age_groups:
    pop_col = f'male_{group}_population_2011'
    density_col = f'male_{group}_per_sq_km_2011'
    df_uk_density_with_gender_and_age_population[density_col] = (
        df_uk_density_with_gender_and_age_population[pop_col] /
        df_uk_density_with_gender_and_age_population['area_sq_km']
    )

# For loop to create a column based on the age group and area_sq_km for females in 2022
for group in age_groups:
    pop_col = f'female_{group}_population_2022'
    density_col = f'female_{group}_per_sq_km_2022'
    df_uk_density_with_gender_and_age_population[density_col] = (
        df_uk_density_with_gender_and_age_population[pop_col] /
        df_uk_density_with_gender_and_age_population['area_sq_km']
    )
# For loop to create a column based on the age group and area_sq_km for females in 2011
for group in age_groups:
    pop_col = f'female_{group}_population_2011'
    density_col = f'female_{group}_per_sq_km_2011'
    df_uk_density_with_gender_and_age_population[density_col] = (
        df_uk_density_with_gender_and_age_population[pop_col] /
        df_uk_density_with_gender_and_age_population['area_sq_km']
    )


**Reviewing the columns of the dataframe**

In [ ]:
df_uk_density_with_gender_and_age_population.columns

**Splitting the dataframe for population density for 2022**

In [ ]:
# Defining the columns based on the information for 2022
df_uk_population_density_2022 = df_uk_density_with_gender_and_age_population[[
    'code', 'name','geography','area_sq_km',
    'population_2022',
    'person_per_sq_km_2022',
    'male_population_2022',
    'female_population_2022',
    'male_per_sq_km_2022',
    'female_per_sq_km_2022',
    'female_early_childhood_population_2022',
    'female_middle_childhood_population_2022',
    'female_teens_population_2022',
    'female_young_adults_population_2022',
    'female_middle_aged_adults_population_2022',
    'female_seniors_elderly_population_2022',
    'male_early_childhood_population_2022',
    'male_middle_childhood_population_2022',
    'male_teens_population_2022',
    'male_young_adults_population_2022',
    'male_middle_aged_adults_population_2022',
    'male_seniors_elderly_population_2022',
    'male_early_childhood_per_sq_km_2022',
    'male_middle_childhood_per_sq_km_2022',
    'male_teens_per_sq_km_2022',
    'male_young_adults_per_sq_km_2022',
    'male_middle_aged_adults_per_sq_km_2022',
    'male_seniors_elderly_per_sq_km_2022',
    'female_early_childhood_per_sq_km_2022',
    'female_middle_childhood_per_sq_km_2022',
    'female_teens_per_sq_km_2022',
    'female_young_adults_per_sq_km_2022',
    'female_middle_aged_adults_per_sq_km_2022',
    'female_seniors_elderly_per_sq_km_2022']]

# Renaming the columns by removing "_2022" in the column name
df_uk_population_density_2022 = df_uk_population_density_2022.rename(
    columns={col: col.replace('_2022', '') for col in df_uk_population_density_2022.columns if '_2022' in col}
)

In [ ]:
# Reviewing the head of the dataframe.
df_uk_population_density_2022.head()

**Saving the dataframe as a CSV file for population density data 2022**

In [ ]:
# Saving the dataframe as df_uk_population_density_2022.csv
df_uk_population_density_2022.to_csv("data/df_uk_population_density_2022.csv", index=False)

**Splitting the dataframe for population density for 2011**

In [ ]:
# Defining the columns based on the information for 2011
df_uk_population_density_2011 = df_uk_density_with_gender_and_age_population[[
    'code', 'name','geography','area_sq_km',
    'population_2011',
    'person_per_sq_km_2011',
    'male_population_2011',
    'female_population_2011',
    'male_per_sq_km_2011',
    'female_per_sq_km_2011',
    'female_early_childhood_population_2011',
    'female_middle_childhood_population_2011',
    'female_teens_population_2011',
    'female_young_adults_population_2011',
    'female_middle_aged_adults_population_2011',
    'female_seniors_elderly_population_2011',
    'male_early_childhood_population_2011',
    'male_middle_childhood_population_2011',
    'male_teens_population_2011',
    'male_young_adults_population_2011',
    'male_middle_aged_adults_population_2011',
    'male_seniors_elderly_population_2011',
    'male_early_childhood_per_sq_km_2011',
    'male_middle_childhood_per_sq_km_2011',
    'male_teens_per_sq_km_2011',
    'male_young_adults_per_sq_km_2011',
    'male_middle_aged_adults_per_sq_km_2011',
    'male_seniors_elderly_per_sq_km_2011',
    'female_early_childhood_per_sq_km_2011',
    'female_middle_childhood_per_sq_km_2011',
    'female_teens_per_sq_km_2011',
    'female_young_adults_per_sq_km_2011',
    'female_middle_aged_adults_per_sq_km_2011',
    'female_seniors_elderly_per_sq_km_2011']]

# Renaming the columns by removing "_2022" in the column name
df_uk_population_density_2011 = df_uk_population_density_2011.rename(
    columns={col: col.replace('_2011', '') for col in df_uk_population_density_2011.columns if '_2011' in col}
)

In [ ]:
# Reviewing the head of the dataframe.
df_uk_population_density_2011.head()

**Saving the dataframe as a CSV file for population density data 2011**

In [ ]:
# Saving the dataframe as df_uk_population_density_2011.csv
df_uk_population_density_2011.to_csv("data/df_uk_population_density_2011.csv", index=False)

# Interactive Visualisation One - Geographical Data Choropleth Map

**Creating the interactive choropleth map**

In [ ]:
%%writefile uk_population_map.py

# Importing Library for uk_population_map.py
import pandas as pd
import plotly.express as px
import streamlit as st
import json

# Loading the dataset based on the uk density for 2022 and 2011
df_uk_density_2022 = pd.read_csv("data/df_uk_population_density_2022.csv")
df_uk_density_2011 = pd.read_csv("data/df_uk_population_density_2011.csv")

# Loading the GeoJSON file for the UK boundaries
with open("data/Local_Authority_Districts_May_2024_Boundaries_UK.geojson", "r") as f:
    geojson = json.load(f)

# Streamlit user input options
st.sidebar.title('Options')
year = st.sidebar.selectbox("Select Year:", [2011, 2022])

# Load the appropriate DataFrame based on the selected year
if year == 2011:
    df_selected = df_uk_density_2011
else:
    df_selected = df_uk_density_2022

# Create the choropleth map
fig = px.choropleth_map(
    df_selected,
    geojson=geojson,
    locations='code',
    color='person_per_sq_km',
    featureidkey="properties.LAD24CD",
    color_continuous_scale=px.colors.sequential.Magma,
    center={"lat": 55.09, "lon": -4.03},
    custom_data=['name','code','population', 'area_sq_km', 'person_per_sq_km'],
    labels={'person_per_sq_km': 'Population<br>Density'},
    zoom=4
)

# Updating the hover template for the choropleth map. 
fig.update_traces(
    hovertemplate="""
    <br><b>%{customdata[0]}</b><br>
    <br><b>Code: </b> %{customdata[1]}
    <br><b>Population: </b> %{customdata[2]}
    <br><b>Area per km²: </b> %{customdata[3]:.0f}
    <br><b>Person per km²: </b> %{customdata[4]:.0f}<br>
    """
)
    
# Updating layout for the choropleth map. 
fig.update_layout(
     title={
        'text': f"UK Population by Local Authority District - {year}",
        'y': 0.95,  # Slightly lower than default
        'x': 0.0,
        'xanchor': 'left',
        'yanchor': 'top'
    },
    hoverlabel=dict(
        bgcolor="white",     
        font_size=14,
        font_color="grey", 
    ),
    margin={"r": 150, "t": 50, "l": 0, "b": 0}
)

# Display the choropleth map
st.plotly_chart(fig, use_container_width=True)

# Execute 'streamlit run uk_population_map.py' on the terminal

# Interactive Visualisation Two - Population Data Bar Chart

In [ ]:
%%writefile uk_population_bar_chart.py

# Importing Library for uk_population_bar_chart.py
import pandas as pd
import plotly.express as px
import streamlit as st

# Loading the dataset based on the uk population
df_uk_population = pd.read_csv("data/df_uk_population.csv")

# Streamlit user input options
st.sidebar.title('Options')
selected_year = st.sidebar.selectbox("Select Year", options=["2011", "2022"], index=1)
selected_gender = st.sidebar.radio("Select Gender", options=["Both", "M", "F"], index=0)

# Selecting appropriate population column based on year
if selected_year == "2022":
    df_uk_population["population"] = df_uk_population["population_2022"]
else:
    df_uk_population["population"] = df_uk_population["population_2011"]

# Selecting appropriate filter based on gender if its a Male or Female
if selected_gender in ["M", "F"]:
    df_uk_population = df_uk_population[df_uk_population["sex"] == selected_gender]

# Grouping the sum values based on age 
df_selected = df_uk_population.groupby("age")["population"].sum().reset_index()

# df_selected = df_selected.sort_values("age")

# Colour mapping based on gender
gender_color_map = {
    "M": "#1f77b4",   # blue
    "F": "#e377c2",   # pink
    "Both": "#7f7f7f"  # grey
}
# Setting the the bar colour based on the gender
bar_color = gender_color_map.get(selected_gender)

# string mapping based on gender
gender_map = {"Both":"Both Genders","M": "Male", "F": "Female"} 
# Setting the the title based on the gender
gender_title = gender_map.get(selected_gender, "Both Genders")


# Creating the bar chart
fig = px.bar(
    df_selected,
    x="age",
    y="population",
    labels={"age": "Age", "population": "Population"},
    title=f"UK Population by Age - {gender_title} ({selected_year})",
    hover_data={"age": True, "population": True}
)

# Updating layout for the bar chart map 
fig.update_layout(bargap=0.2)

# Updating colour for bar chart map 
fig.update_traces(marker_color=bar_color)

# Display the bar map
st.plotly_chart(fig, use_container_width=True)

# Execute 'streamlit run uk_population_bar_chart.py' on the terminal

# The Final Dashboard - Interactive Visualisation

In [1]:
%%writefile dashboard_interactive_visualisation.py
# Importing Library for dashboard_interactive_visualisation.py
import pandas as pd
import plotly.express as px
import streamlit as st
import json

# Setting the page layout
st.set_page_config(layout="wide")

# Loading the datasets for 2022 and 2011
df_uk_density_2022 = pd.read_csv("data/df_uk_population_density_2022.csv")
df_uk_density_2011 = pd.read_csv("data/df_uk_population_density_2011.csv")

# Loading the GeoJSON file for the UK boundaries
with open("data/Local_Authority_Districts_May_2024_Boundaries_UK.geojson", "r") as f:
    geojson = json.load(f)

######  Streamlit Sidebar Controls Start ######

# Setting sidebar title
st.sidebar.title("UK Population Density Map")

# Year selection (2022 or 2011)
year_options = [2022, 2011]
selected_year = st.sidebar.selectbox("Select Year", year_options)

# Gender selection
gender_options = ['Both Genders', 'Male', 'Female']
selected_gender = st.sidebar.selectbox("Select Gender", gender_options)

# Age group label-to-key mapping
age_group_map = {
    'All Ages': 'All',
    'Early Childhood (0–5)': 'early_childhood',
    'Middle Childhood (6–12)': 'middle_childhood',
    'Teens (13–18)': 'teens',
    'Young Adults (19–39)': 'young_adults',
    'Middle Aged Adults (40–59)': 'middle_aged_adults',
    'Seniors/Elderly (60+)': 'seniors_elderly'
}

# Age group selection
age_group_option = st.sidebar.selectbox("Select Age Group", list(age_group_map.keys()))
selected_age_group = age_group_map[age_group_option]

######  Streamlit Sidebar Controls End ######

###### Helper Functions for Options Start ######

# Function to get the correct density column based on gender and age group
def get_density_column(gender, age_group):
    if gender == 'Both Genders':
        return 'person_per_sq_km'
    elif age_group == 'All':
        return f"{gender.lower()}_per_sq_km"
    else:
        return f"{gender.lower()}_{age_group.lower()}_per_sq_km"

# Function to get the correct population column based on gender, age group and dataframe
def get_population_column(gender, age_group, df):
    if gender == 'Both Genders':
        if age_group == 'All':
            return 'population'
        else:
            male_col = f"male_{age_group}_population"
            female_col = f"female_{age_group}_population"
            df['combined_population'] = df[male_col] + df[female_col]
            return 'combined_population'
    else:
        if age_group == 'All':
            return f"{gender.lower()}_population"
        else:
            return f"{gender.lower()}_{age_group}_population"

###### Helper Functions for Options End ######

###### Data Selection Start ######

# Select the correct dataframe based on the selected year
if selected_year == 2022:
    df_selected = df_uk_density_2022
else:
    df_selected = df_uk_density_2011

# Get the correct density and population columns
density_column = get_density_column(selected_gender, selected_age_group)
population_column = get_population_column(selected_gender, selected_age_group, df_selected)

# Population label for the selected columns
if selected_gender == 'Both Genders' and selected_age_group != 'All':
    population_label = "Population"
else:
    population_label = population_column.replace('_', ' ').title()

# Adjust custom_data based on the population_column 
custom_data = ['name', 'code', 'area_sq_km', population_column]

###### Data Selection Start ######

###### Create Choropleth Map Start ######

# Create the choropleth map
fig = px.choropleth_map(
    df_selected,
    geojson=geojson,
    locations='code',
    color=density_column,
    featureidkey="properties.LAD24CD",
    color_continuous_scale=px.colors.sequential.Magma,
    center={"lat": 55.09, "lon": -4.03},
    custom_data=custom_data,
    labels={density_column:'Population<br>Density'},
    zoom=4.8
)

# Updating the hover template for the choropleth map
fig.update_traces(
    hovertemplate=f"""
    <br><b>%{{customdata[0]}}</b><br>
    <br><b>Code: </b> %{{customdata[1]}}
    <br><b>Area per km²: </b> %{{customdata[2]:.0f}}
    <br><b>{population_label}: </b> %{{customdata[3]:.0f}}<br>
    """
)

# Updating layout for the choropleth map
fig.update_layout(
    height=800,
    title={
        'text': f"UK Population Density by Local Authority District {selected_year} - {selected_gender} - {age_group_option}",
        'xanchor': 'left',
        'yanchor': 'top'
    },
    hoverlabel=dict(
        bgcolor="white",
        font_size=14,
        font_color="grey", 
    ),
    margin={"r": 150, "t": 50, "l": 0, "b": 0}
)

###### Create Choropleth Map End ######

# Display the choropleth map
st.plotly_chart(fig, use_container_width=True)

# Execute 'streamlit run dashboard_interactive_visualisation.py' on the terminal

Overwriting dashboard_interactive_visualisation.py


In [ ]:
!streamlit run dashboard_interactive_visualisation.py